In [18]:
import math

import duckdb
import plotly.express as px
import plotly.graph_objects as go

from utils import  wrap_to_string,trimmer

conn = duckdb.connect("dataset.db")

# Structural Investigation

Let's look at the general structure of the dataset

In [3]:
query = "SELECT COUNT(column_name) FROM information_schema.columns WHERE table_name='dataset'"
conn.sql(query).show()

┌────────────────────┐
│ count(column_name) │
│       int64        │
├────────────────────┤
│                 67 │
└────────────────────┘



In [4]:
query  = "SELECT COUNT(*) FROM dataset"
conn.sql(query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       363243 │
└──────────────┘



Count how many times each data type is present in the dataset

In [5]:
query = "SELECT data_type,COUNT(*) FROM information_schema.columns GROUP BY data_type"
conn.sql(query).show()

┌───────────┬──────────────┐
│ data_type │ count_star() │
│  varchar  │    int64     │
├───────────┼──────────────┤
│ VARCHAR   │            6 │
│ BIGINT    │            6 │
│ DOUBLE    │           55 │
└───────────┴──────────────┘



## 1.1. Structure of non-numerical features

In [6]:
query = """
SELECT column_name
FROM information_schema.columns
WHERE data_type NOT IN ('BIGINT', 'DOUBLE')
"""
conn.sql(query).show()

┌───────────────────────────┐
│        column_name        │
│          varchar          │
├───────────────────────────┤
│ Accident_Index            │
│ Sex_of_Driver             │
│ Date                      │
│ Time                      │
│ Local_Authority_(Highway) │
│ LSOA_of_Accident_Location │
└───────────────────────────┘



We have all the necessary column names from the above table. Unfortunately all my attempts to transform the above results to columns results in catastrophically bad looking and unintuitve code. So we'll tamely switch to python for a moment

In [7]:
query = """
SELECT column_name
FROM information_schema.columns
WHERE data_type NOT IN ('BIGINT', 'DOUBLE')
"""
df = conn.sql(query).df()
non_numeric_columns =  wrap_to_string(df)

# DcukDB has a nice summarize command to help out here
query = f"""
SUMMARIZE SELECT {non_numeric_columns} FROM dataset
"""
conn.sql(query).show()

┌──────────────────────┬─────────────┬───────────────┬───────────────┬───┬───────┬───────┬────────┬─────────────────┐
│     column_name      │ column_type │      min      │      max      │ … │  q50  │  q75  │ count  │ null_percentage │
│       varchar        │   varchar   │    varchar    │    varchar    │   │ int32 │ int32 │ int64  │     varchar     │
├──────────────────────┼─────────────┼───────────────┼───────────────┼───┼───────┼───────┼────────┼─────────────────┤
│ Accident_Index       │ VARCHAR     │ 201501BS70001 │ 2015984141415 │ … │  NULL │  NULL │ 363243 │ 0.0%            │
│ Sex_of_Driver        │ VARCHAR     │ 1.0           │ 3.0           │ … │  NULL │  NULL │ 363243 │ 0.0%            │
│ Date                 │ VARCHAR     │ 01/01/2015    │ 31/12/2015    │ … │  NULL │  NULL │ 363243 │ 11.94%          │
│ Time                 │ VARCHAR     │ 00:01         │ 23:59         │ … │  NULL │  NULL │ 363243 │ 11.95%          │
│ Local_Authority_(H…  │ VARCHAR     │ E06000001     │ W

## 1.2. Structure of numerical features

In [8]:
query = """
SELECT column_name
FROM information_schema.columns
WHERE data_type IN ('BIGINT', 'DOUBLE')
"""
numeric_columns =  wrap_to_string(conn.sql(query).df())

In [9]:
query = f"""
SELECT DISTINCT {numeric_columns} FROM dataset"""
conn.sql(query).df()

,Vehicle_Reference_df_res,Vehicle_Reference_df,Casualty_Reference,Casualty_Class,Casualty_Severity,Casualty_Type,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,...,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,1,2,2,2,3,9,9.0,0.0,18.0,0.0,...,1.0,16.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0,8.0
1,1,1,1,3,3,0,9.0,0.0,4.0,0.0,...,1.0,28.0,6.0,9.0,5.0,0.0,0.0,0.0,1.0,10.0
2,2,2,1,1,3,1,1.0,0.0,17.0,0.0,...,1.0,68.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0
3,1,2,2,2,3,9,9.0,0.0,7.0,0.0,...,2.0,57.0,9.0,0.0,0.0,1.0,0.0,0.0,3.0,9.0
4,1,2,1,1,3,1,9.0,0.0,9.0,0.0,...,1.0,34.0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363216,2,2,2,1,3,9,9.0,0.0,9.0,0.0,...,1.0,21.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
363217,1,2,1,1,3,9,9.0,0.0,18.0,0.0,...,2.0,45.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0
363218,1,1,1,3,3,0,9.0,0.0,18.0,0.0,...,1.0,27.0,6.0,1.0,1.0,0.0,0.0,0.0,1.0,10.0
363219,2,1,1,1,3,9,9.0,0.0,3.0,0.0,...,1.0,44.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


Doing a distinct Count of these columns involves creating abominations like [this](https://stackoverflow.com/a/1363602/14274188). I don't want that and so am tamely using Python instead

In [29]:
distinct_counts = conn.sql(query).df().count().sort_values()

fig = go.Figure()
fig.add_trace(go.Bar(y=distinct_counts,x=list(map(trimmer,distinct_counts.index))))
fig.show()